Source: https://www.kaggle.com/giuseppemerendino/deep-mushroom-keras-t-sne

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math

In [18]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from sklearn.model_selection import cross_val_score
from keras import backend as K
from keras.layers import BatchNormalization

C:\Users\Duo\Anaconda3\envs\fastai\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [19]:
seed = 123456

In [4]:
path = "data\\sorted\\agg\\ids.csv"

data = pd.read_csv(path)

In [11]:
data.head()

,name,date,flow_rate,source,peak_val,area,volume,eccentricity,euler_number,extent,mean_intensity,moments,orientation,perimeter,inertia_tensor
0,0,2017.11.09,1800,chip2_02heightmap.mat,58.496427,4489,152983.389556,0.866207,0,0.673316,34.079615,[[4.48900000e+03 2.35521000e+05 1.56682350e+07...,1.309976,365.220346,[[232.59219643 145.14005797]\r\n [145.14005797...
1,1,2017.11.09,1800,chip2_03heightmap.mat,5.237538,6320,10642.551390,0.871961,-1,0.383273,1.682883,[[6.32400000e+03 6.59633000e+05 7.81561010e+07...,-1.266807,732.523953,[[ 488.4762157 -344.60676222]\r\n [-344.60676...
2,2,2017.11.09,1800,chip2_04heightmap.mat,16.623206,443,3682.088758,0.902466,1,0.692187,8.311713,[[4.43000000e+02 7.34200000e+03 1.61000000e+05...,1.470180,94.526912,[[17.34078645 7.28397087]\r\n [ 7.28397087 88...
3,3,2017.11.09,1800,chip2_05heightmap.mat,12.465707,1054,6977.762945,0.615661,1,0.552411,6.620268,[[1.05400000e+03 2.80700000e+04 8.86760000e+05...,-1.477930,148.225397,[[ 82.71136252 -4.63732992]\r\n [ -4.63732992...
4,4,2017.11.09,1800,chip2_06heightmap.mat,4.629928,1929,3566.980825,0.871295,1,0.562063,1.849135,[[1.92900000e+03 6.37560000e+04 2.84693000e+06...,-1.543902,209.095454,[[ 92.6177739 -7.82965009]\r\n [ -7.82965009...


In [12]:
data.shape

(402, 15)

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402 entries, 0 to 401
Data columns (total 15 columns):
name              402 non-null object
date              402 non-null object
flow_rate         402 non-null int64
source            402 non-null object
peak_val          402 non-null float64
area              402 non-null int64
volume            402 non-null float64
eccentricity      402 non-null float64
euler_number      402 non-null int64
extent            402 non-null float64
mean_intensity    402 non-null float64
moments           402 non-null object
orientation       402 non-null float64
perimeter         402 non-null float64
inertia_tensor    402 non-null object
dtypes: float64(7), int64(3), object(5)
memory usage: 47.2+ KB


In [15]:
predictors = data.drop(columns=['name','date','flow_rate','source','moments','inertia_tensor'])

In [16]:
predictors.head()

,peak_val,area,volume,eccentricity,euler_number,extent,mean_intensity,orientation,perimeter
0,58.496427,4489,152983.389556,0.866207,0,0.673316,34.079615,1.309976,365.220346
1,5.237538,6320,10642.551390,0.871961,-1,0.383273,1.682883,-1.266807,732.523953
2,16.623206,443,3682.088758,0.902466,1,0.692187,8.311713,1.470180,94.526912
3,12.465707,1054,6977.762945,0.615661,1,0.552411,6.620268,-1.477930,148.225397
4,4.629928,1929,3566.980825,0.871295,1,0.562063,1.849135,-1.543902,209.095454


In [17]:
Y = pd.get_dummies(data.iloc[:,0],  drop_first=False)
X = pd.DataFrame()
for each in data.iloc[:,1:].columns:
    dummies = pd.get_dummies(data[each], prefix=each, drop_first=False)
    X = pd.concat([X, dummies], axis=1)

In [20]:
def create_model():
    model = Sequential()
    model.add(Dense(20, input_dim=X.shape[1], kernel_initializer='uniform', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    #model.add(Dense(20, input_dim=X.shape[1], kernel_initializer='uniform', activation='relu'))
    #model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    #sgd = SGD(lr=0.01, momentum=0.7, decay=0, nesterov=False)
    model.compile(loss='binary_crossentropy' , optimizer='adam', metrics=['accuracy'])
    return model

In [21]:
model = create_model()
history = model.fit(X.values, Y.values, validation_split=0.50, epochs=100, batch_size=50, verbose=0)


# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

ValueError: Error when checking target: expected dense_2 to have shape (2,) but got array with shape (402,)